In [1]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
%matplotlib inline

import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk import ngrams

np.random.seed(42)
random.seed(42)

In [2]:
#загружаем словарь сильных и слабых слов, где только униграммы

dictionary = pd.read_excel('dict_unigrams.xlsx')
dictionary

,weak,strong
0,почти,определенно
1,практически,явно
2,приблизительно,явный
3,приближенно,однозначно
4,видимо,отчетливо
5,вероятно,точно
6,похоже,наибольший
7,кажется,наименьший
8,предполагаемый,должен
9,неопределенный,никогда


In [3]:
#преобразуем колонки в списки

weak = dictionary['weak'].tolist()
strong = dictionary['strong'].tolist()

In [4]:
m = MorphAnalyzer()
regex = re.compile("[А-Яа-я]+")

#очищаем текст + токенизация
def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

#лемматизация токена
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]

#общая функция
def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    return lemmas

Работаем с дата-сетом ПРЕСС-РЕЛИЗОВ по ставке

In [5]:
pr = pd.read_excel('!PR_datasetN4.xls')
pr.head()

,Date,Text
0,2015-01-30,Совет директоров Банка России 30 января 2015 г...
1,2015-03-13,Совет директоров Банка России 13 марта 2015 го...
2,2015-04-30,Совет директоров Банка России 30 апреля 2015 г...
3,2015-06-15,Совет директоров Банка России 15 июня 2015 год...
4,2015-07-31,Совет директоров Банка России 31 июля 2015 год...


In [6]:
#лемматизируем пресс-релизы

lemmas = list(pr['Text'].apply(clean_text))
pr['lemmas'] = lemmas
pr.head()

,Date,Text,lemmas
0,2015-01-30,Совет директоров Банка России 30 января 2015 г...,"[совет, директор, банк, россия, январь, год, п..."
1,2015-03-13,Совет директоров Банка России 13 марта 2015 го...,"[совет, директор, банк, россия, март, год, при..."
2,2015-04-30,Совет директоров Банка России 30 апреля 2015 г...,"[совет, директор, банк, россия, апрель, год, п..."
3,2015-06-15,Совет директоров Банка России 15 июня 2015 год...,"[совет, директор, банк, россия, июнь, год, при..."
4,2015-07-31,Совет директоров Банка России 31 июля 2015 год...,"[совет, директор, банк, россия, июль, год, при..."


In [7]:
#создадим списки с количеством слабых и сильных слов, которые встретились в пресс-релизах

weak_pr, strong_pr = [], []
count_weak, count_strong = 0, 0

for row in pr['lemmas']:
    for word in row:
        if word in weak:
            count_weak += 1
        elif word in strong:
            count_strong += 1
    weak_pr.append(count_weak)
    strong_pr.append(count_strong)
    count_weak, count_strong = 0, 0
    
print(weak_pr)
print(strong_pr)

[3, 1, 3, 3, 2, 1, 1, 4, 2, 3, 1, 1, 2, 2, 2, 3, 2, 3, 5, 1, 1, 3, 0, 5, 5, 3, 4, 6, 3, 5, 4, 5, 4, 4, 5, 3, 2, 4, 5, 5, 5, 2, 3, 5, 2, 9, 9, 4, 5, 3, 2, 3, 4, 5, 3, 1, 1, 0, 0, 0, 2, 1, 1, 0, 3, 6]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
print('Всего слабых слов в пресс-релизах:', sum(weak_pr))
print('Всего сильных слов в пресс-релизах:', sum(strong_pr))

Всего слабых слов в пресс-релизах: 200
Всего сильных слов в пресс-релизах: 1


In [9]:
pr['weak'] = weak_pr
pr['strong'] = strong_pr

In [12]:
pr['formulaic'] = (pr['weak']-pr['strong'])/(pr['weak']+pr['strong'])

In [18]:
pr['formulaic'] = pr['formulaic'].fillna(0)

In [19]:
pr.head()

,Date,Text,lemmas,weak,strong,formulaic
0,2015-01-30,Совет директоров Банка России 30 января 2015 г...,"[совет, директор, банк, россия, январь, год, п...",3,0,1.0
1,2015-03-13,Совет директоров Банка России 13 марта 2015 го...,"[совет, директор, банк, россия, март, год, при...",1,0,1.0
2,2015-04-30,Совет директоров Банка России 30 апреля 2015 г...,"[совет, директор, банк, россия, апрель, год, п...",3,0,1.0
3,2015-06-15,Совет директоров Банка России 15 июня 2015 год...,"[совет, директор, банк, россия, июнь, год, при...",3,0,1.0
4,2015-07-31,Совет директоров Банка России 31 июля 2015 год...,"[совет, директор, банк, россия, июль, год, при...",2,0,1.0


In [20]:
pr.to_excel('pr_formulaic_unigrams.xls')

In [28]:
#загружаем датасет заявлений по ставке

zp = pd.read_excel('!ZP_DATASET.xls')
zp.head()

,Date,Text
0,2015-03-13,Сегодня Совет директоров Банка России принял р...
1,2015-06-15,Сегодня Совет директоров Банка России принял р...
2,2015-09-11,Сегодня Банк России принял решение сохранить к...
3,2015-12-11,Сегодня Банк России принял решение сохранить к...
4,2016-03-18,Сегодня Банк России принял решение сохранить к...


In [29]:
#лемматизируем заявления

lemmas = list(zp['Text'].apply(clean_text))
zp['lemmas'] = lemmas
zp.head()

,Date,Text,lemmas
0,2015-03-13,Сегодня Совет директоров Банка России принял р...,"[сегодня, совет, директор, банк, россия, приня..."
1,2015-06-15,Сегодня Совет директоров Банка России принял р...,"[сегодня, совет, директор, банк, россия, приня..."
2,2015-09-11,Сегодня Банк России принял решение сохранить к...,"[сегодня, банк, россия, принять, решение, сохр..."
3,2015-12-11,Сегодня Банк России принял решение сохранить к...,"[сегодня, банк, россия, принять, решение, сохр..."
4,2016-03-18,Сегодня Банк России принял решение сохранить к...,"[сегодня, банк, россия, принять, решение, сохр..."


In [32]:
#создадим списки с количеством слабых и сильных слов, которые встретились в заявлениях по ставке

weak_zp, strong_zp = [], []
count_weak, count_strong = 0, 0

for row in zp['lemmas']:
    for word in row:
        if word in weak:
            count_weak += 1
        elif word in strong:
            count_strong += 1
    weak_zp.append(count_weak)
    strong_zp.append(count_strong)
    count_weak, count_strong = 0, 0
    
print(weak_zp)
print(strong_zp)

[5, 11, 12, 14, 16, 7, 8, 7, 10, 9, 8, 8, 5, 9, 4, 7, 9, 8, 4, 7, 12, 3, 6, 8, 3, 7, 12, 3, 5, 4, 3, 2, 7, 7, 3, 5, 3, 2, 3, 5, 8, 6, 7, 7]
[0, 1, 1, 1, 0, 1, 1, 2, 2, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 1, 0, 0, 2, 0, 0, 1, 0, 1, 0, 1, 2, 2, 0, 1, 0, 2, 1, 2, 1]


In [31]:
print('Всего слабых слов в заявлениях:', sum(weak_zp))
print('Всего сильных слов в заявлениях:', sum(strong_zp))

Всего слабых слов в заявлениях: 299
Всего сильных слов в заявлениях: 34
